In [1]:
import hashlib
import json
import requests
import pandas as pd



URL = 'https://scanner.tradingview.com/crypto/scan'
body = {
    "filter":[],
    "options":{
        "lang":"en"
    },
    "markets":[
        "crypto"
    ],
    "symbols":{
        "query":{
            "types":[]
        },
        "tickers":[]
    },
    "columns":[
        "name",
        "base_currency",
        "currency",
        "close",
        "exchange",
        "type",
        "subtype"
    ],
    "sort":{
        "sortBy":"24h_vol|5",
        "sortOrder":"desc"
    },
    "price_conversion":{
        "to_symbol":False
    },
    "range":[0,100000]
}


d = pd.DataFrame()
res = requests.post(URL, data=json.dumps(body))

df = dict()
if res.status_code == 200:
    response = json.loads(res.text)
    count = response['totalCount']
    data = [d['d'] if d['d'][1] and d['d'][2] else [] for d in response['data']]
    df = pd.DataFrame(
        data, 
        columns=['NAME', 'CUR1', 'CUR2', 'PRICE1', 'EXCH', 'TYPE', 'SUBTYPE']
    )


In [2]:
exch = [
    'UNISWAP', 'BINANCE', 'MEXC', 'GATEIO',
    'HITBTC', 'UNISWAP3ETH', 'KUCOIN', 'BITTREX',
    'COINEX', 'OKX', 'HUOBI', 'KRAKEN',
    'SUSHISWAP', 'POLONIEX', 'BITMEX', 'BITFINEX',
    'COINBASE', 'BITGET', 'UNISWAP3POLYGON', 'PHEMEX',
    'BITRUE', 'BINANCEUS', 'UPBIT', 'BYBIT',
    'WHITEBIT', 'BITHUMB', 'MERCADO'
]

print(len(df))

# Remove emty rows
df.dropna(subset=['NAME'], inplace=True)

# Remove undesible exchanges
df = df[df.EXCH.isin(exch)]

# Remove where price is 0
df = df[df.PRICE1 != 0]

# Remove rows where CUR1 == CUR2
df = df[df.CUR1 != df.CUR2]

# Remove rows with special pairs
df = df[df.CUR1 + df.CUR2 == df.NAME ]

# Create new col price of reverse currency
df['PRICE2'] = 1 / df['PRICE1']

def hash_unicode(a_string):
    return hashlib.sha256(a_string.encode('utf-8')).hexdigest()

df['HASH'] = df['NAME'] + df['EXCH'] + df['TYPE']
df['HASH'] = df['HASH'].apply(hash_unicode)

print(len(df))
# df.to_csv('./test.csv', sep='\t')

39257
6784


In [14]:
len('6e0e685e37d37476ee06636207a6f44b5f7d7d17c62a281758536b14df5355b2')

64

In [79]:


# Create reverse CUR1 and CUR2
reverse_df = df.copy()
reverse_df['CUR1'] = df['CUR2']
reverse_df['CUR2'] = df['CUR1']
reverse_df['PRICE1'] = df['PRICE2']
reverse_df['PRICE2'] = df['PRICE1']


# Concatanete df
df = pd.concat([df,reverse_df])
print(len(df))


13532


In [80]:
y_df = df.merge(
    df,
    left_on='CUR1',
    right_on='CUR2'
)

    
# Create single pair arbitrage table
single_df = y_df[y_df.NAME_x == y_df.NAME_y]
single_df = single_df[single_df.EXCH_x != single_df.EXCH_y]

single_df['PROF1'] = round((single_df['PRICE2_y'] * 100 / single_df['PRICE1_x'])-100, 5)
single_df['PROF2'] = round((single_df['PRICE2_y'] * 100 / single_df['PRICE1_x'])-100, 5)


# # # Sort result by profit
single_df = single_df[
    ((single_df.PROF1 >= 1) | 
    (single_df.PROF2 >= 1)) & 
    ((single_df.PROF1 <= 100) |
     (single_df.PROF2 <= 100))
].sort_values('PROF1', ascending=False)
print('Single: ', len(single_df))
single_df

Single:  664


,NAME_x,CUR1_x,CUR2_x,PRICE1_x,EXCH_x,TYPE_x,SUBTYPE_x,PRICE2_x,NAME_y,CUR1_y,CUR2_y,PRICE1_y,EXCH_y,TYPE_y,SUBTYPE_y,PRICE2_y,PROF1,PROF2
6058794,QIUSDT,USDT,QI,58.343057,MEXC,spot,crypto,0.017140,QIUSDT,QI,USDT,0.008790,BINANCE,crypto,defi,113.765643,94.99431,94.99431
12273148,QIUSDT,QI,USDT,0.008790,BINANCE,crypto,defi,113.765643,QIUSDT,USDT,QI,58.343057,MEXC,spot,crypto,0.017140,94.99431,94.99431
12288018,DFLUSDT,DFL,USDT,0.001467,MEXC,spot,crypto,681.663258,DFLUSDT,USDT,DFL,354.735722,GATEIO,spot,crypto,0.002819,92.16087,92.16087
9451034,DFLUSDT,USDT,DFL,354.735722,GATEIO,spot,crypto,0.002819,DFLUSDT,DFL,USDT,0.001467,MEXC,spot,crypto,681.663258,92.16087,92.16087
12273149,QIUSDT,QI,USDT,0.008790,BINANCE,crypto,defi,113.765643,QIUSDT,USDT,QI,60.532688,GATEIO,spot,crypto,0.016520,87.94084,87.94084
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12289912,CRAFTUSDT,CRAFT,USDT,0.020878,GATEIO,spot,crypto,47.897308,CRAFTUSDT,USDT,CRAFT,47.415837,MEXC,spot,crypto,0.021090,1.01542,1.01542
8615830,ARGUSDT,USDT,ARG,0.678426,MEXC,spot,crypto,1.474000,ARGUSDT,ARG,USDT,1.459200,GATEIO,spot,crypto,0.685307,1.01425,1.01425
12289292,ARGUSDT,ARG,USDT,1.459200,GATEIO,spot,crypto,0.685307,ARGUSDT,USDT,ARG,0.678426,MEXC,spot,crypto,1.474000,1.01425,1.01425
12283402,RLYUSDT,RLY,USDT,0.015145,MEXC,spot,crypto,66.028392,RLYUSDT,USDT,RLY,65.372295,GATEIO,spot,crypto,0.015297,1.00363,1.00363


In [81]:
''' Cross-exchange arbitrage multiple pairs'''
import sqlalchemy

engine = sqlalchemy.create_engine('sqlite:///arbitrage.db')
single_df.to_sql('single_arbitrage', engine, if_exists='replace', index=True)

664

In [ ]:
x_df = df.merge(
  df,
  left_on='BASE_CUR',
  right_on='CUR'
)

x_df = df.merge(
  df,
  left_on='CUR',
  right_on='BASE_CUR'
)


y_df = df.merge(
  df,
  left_on='CUR',
  right_on='BASE_CUR'
)




# Create single pair arbitrage table
single_df = y_df[y_df.NAME_x == y_df.NAME_y]
single_df['PROF'] = round((single_df['PRICE_y'] * 100 / single_df['PRICE_x'])-100, 5)

# Sort result by profit
single_df = single_df[single_df.PROF >= 1].sort_values('PROF', ascending=False)
print('Single: ', len(single_df))


print(len(y_df))

,NAME_x,BASE_CUR_x,CUR_x,PRICE_x,EXCH_x,TYPE_x,SUBTYPE_x,NAME_y,BASE_CUR_y,CUR_y,PRICE_y,EXCH_y,TYPE_y,SUBTYPE_y
